# Custom Callbacks

## Configuration

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

# Make plots live-update while scans run.
from bluesky.utils import install_nb_kicker
install_nb_kicker()

import numpy as np
from bluesky import RunEngine
from bluesky.plans import scan
from bluesky.examples import motor, noisy_det

RE = RunEngine({})

## Data Acquisition

### Simple Custom Callbacks

In [ ]:
# No callbacks
RE(scan([noisy_det], motor, 1, 5, 5))

In [ ]:
# Print the type of document as each one is emitted.

def cb(name, doc):
    print(name)

RE(scan([noisy_det], motor, 1, 5, 5), cb)

In [ ]:
# Print the event documents as they are emitted.

def cb(name, doc):
    if name == 'event':
        print(doc)

RE(scan([noisy_det], motor, 1, 5, 5), cb)

In [ ]:
# Make a simple table with sequence number, motor setpoint, and detector reading.

def cb(name, doc):
    if name == 'event':
        print(doc['seq_num'], doc['data']['motor_setpoint'], doc['data']['noisy_det'])

RE(scan([noisy_det], motor, 1, 5, 5), cb)

In [ ]:
# Compute the duration between the 'start' document and the 'stop' document.
# Version 1 -- using a global variable (simple, but not recommended)

start_time = None

def cb(name, doc):
    global start_time
    if name == 'start':
        start_time = doc['time']
    if name == 'stop':
        stop_time = doc['time']
        duration = stop_time - start_time
        print('Duration:', duration, 'seconds')

RE(scan([noisy_det], motor, 1, 5, 5), cb)

In [ ]:
# Compute the duration between the 'start' document and the 'stop' document.
# Version 2 -- using a class variable (recommended)

from bluesky.callbacks import CallbackBase


class Timer(CallbackBase):
    def start(self, doc):
        self.start_time = doc['time']
    def stop(self, doc):
        stop_time = doc['time']
        duration = stop_time - self.start_time
        print('Duration:', duration, 'seconds')

RE(scan([noisy_det], motor, 1, 5, 5), Timer())

### Binding Callbacks to Plans

In [ ]:
from bluesky.plans import subs_decorator


@subs_decorator(cb)
def my_scan(detectors, motor, start, stop, num):
    yield from scan(detectors, motor, start, stop, num)

RE(my_scan([noisy_det], motor, 1, 5, 5))

In [ ]:
# A useful pattern: Pass positional and keyword arguments through, blindly.

@subs_decorator(cb)
def my_scan(*args, **kwargs):
    yield from scan(*args, **kwargs)

RE(my_scan([noisy_det], motor, 1, 5, 5))

## Exercises

1. Write a custom callback that prints the detector reading divided by the motor position.
2. Use `subs_decorator` to bind it to a custom variant of the `scan` plan.
3. Write a custom callback that prints the contents of the start document. Then, if you can, make the output more readable by put each key/value pair on its own line.